In [10]:
import random
import sys
sys.setrecursionlimit(2000)

"""
GCD 함수 구현
"""
def gcd(a, b):
    if a < b:
        a, b = b, a
    if a == b:
        return a
    elif b == 0:
        return a
    else:
        return gcd(b, a % b)


"""
확장 유클리드 알고리즘
"""
def extended_euclid(a, b):

    if a == b:
        return 1, 0, a
    elif b == 0:
        return 1, 0, a
    else:
        x_1 = 1
        y_1 = 0
        r_1 = a

        x_2 = 0
        y_2 = 1
        r_2 = b

        while r_2 != 0:
            q = r_1 // r_2

            r_t = r_1 - q * r_2
            x_t = x_1 - q * x_2
            y_t = y_1 - q * y_2

            x_1, y_1, r_1 = x_2, y_2, r_2
            x_2, y_2, r_2 = x_t, y_t, r_t

        return x_1, y_1, r_1


"""
Multiplicative Inverse 구현

"""
def m_inv(a, n):
    x, y, r = extended_euclid(n, a % n)
    if r != 1:
        print("multiplicative inverse가 없습니다 \n")
        return
    else:
        return y % n

"""
Mille,Rabin_Test
"""
Prime = 0
Composite = 1
def miller_rabin(n, s):
    if n == 2:
        return Prime
    elif n % 2 == 0:
        return Composite

    for _ in range(s):
        a = random.randint(1, n-1)
        if test(a, n) == True:
            return Composite

    return Prime

"""
Miller-Rabin_Test2 
"""
def test(a, n):
    bits = int_to_bin(n-1)
    k = len(bits) - 1
    t = 0

    while bits[k] == '0':
        t += 1
        k -= 1

    u = (n-1) >> t
    x = exp(a, u, n)
    for _ in range(t):
        _x = x
        x = (_x * _x) % n
        if x == 1 and _x != 1 and _x != n-1:
            return True

    if x != 1:
        return True
    else:
        return False

"""
정수를 이진수로 변환하는 함수다
주어진 정수를 이진수 형태의 리스토 변환
"""
def int_to_bin(num):
    return list(bin(num))[2:]  # eliminate first two, ex) 0b00101001

"""
모듈러 지수함수 구현
a^b mod n
"""
def exp(a, b, n):
    c = 0
    f = 1
    bin_b = int_to_bin(b)
    k = len(bin_b)
    for i in range(k):   
        c = 2*c
        f = f*f % n
        if bin_b[i] == '1':
            c = c+1
            f = f*a % n
    return f

"""
공개키와 개인키를 생성
keylen에 따라 p와 q를 생성하고, 이를 사용하여 공개 키, 개인 키, 그리고 n 계산
"""
def keygen(keylen):
    bound = 1 << keylen//2   # upper bound for p and q.
    p = 2 * random.randint(bound//4, bound//2) - 1       # guarantee that p is odd.
    while miller_rabin(p, 50) == Composite:
        p = 2 * random.randint(bound//4, bound//2) - 1   # select new odd p.
    q = 2 * random.randint(bound//4, bound//2) - 1       # guarantee that q is odd.
    while miller_rabin(q, 50) == Composite:
        q = 2 * random.randint(bound//4, bound//2) - 1   # select new odd q.
    # Now p and q are odd primes.

    n = p * q
    phi_n = (p-1) * (q-1)  # euler totient function phi(n)

    # find e, satisfing gcd(phi(n), e) == 1, where 1 < e < phi(n)
    while gcd(phi_n, (e := random.randint(3, phi_n))) != 1:
        pass
    d = m_inv(e, phi_n)  # multiplicative inverse of Modular phi(n)

    return (e, d, n)

"""
RSA 암호화
평문 M, 공개키 e, n을 사용해 암호문 계산
"""
def encrypt(M, e, n):
    return exp(M, e, n)

"""
RSA 복호화
암호문 c, 개인키 d, n을 사용하여 평문 개산
"""
def decrypt(C, d, n):
    return exp(C, d, n)

"""
RSA test
"""
if __name__ == "__main__":
    e, d, n = keygen(512)
    print("공개키 : ", e )
    print("\n")
    M = int(input('평문을 숫자로 적어주세요 : '))
    print("\n")
    C = encrypt(M, e, n)
    MM = decrypt(C, d, n)
    if M == MM:
        print("RSA가 성공적으로 생성되었습니다 \n")
        print("평문={}\n\n공개키=({},{})\n\n개인키=({},{})\n\n암호문={}\n\n복호화된 평문={} \n".format(M, e, n, d, n, C, MM))
    else:
        print("RSA 생성이 잘 안되었습니다")
        print("평문={}\n\n공개키=({},{})\n\n개인키=({},{})\n\n암호문={}\n\n복호화된 평문={} \n".format(M, e, n, d, n, C, MM))

공개키 :  12426264125659081471023322341873134712140048519769578748896480414988503777736169494035323707643112120969523290413706547771599831561027238661969452552770959


평문을 숫자로 적어주세요 : 100


RSA가 성공적으로 생성되었습니다 

평문=100

공개키=(12426264125659081471023322341873134712140048519769578748896480414988503777736169494035323707643112120969523290413706547771599831561027238661969452552770959,13167279230968646193293570613156645453603421614661909714525178122480659415685302745231242825167451635598226608400891813033959798314250075435520383437408019)

개인키=(9583584774564310913137999162394393029766143272297028300992292701872325370292183697335847598209342188388504851817572574510649394727479891329496559265593199,13167279230968646193293570613156645453603421614661909714525178122480659415685302745231242825167451635598226608400891813033959798314250075435520383437408019)

암호문=40841165455072553202000445422847066943897588361157102043057354309400794947671642672113430312837267272245600906194650548649995548612013669846909